In [1]:
import re
import csv
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

In [2]:
def read_wkt(wkt_str):
    """Read wkt polygons 
    
    Args:
        wkt_str (str) : wkt Polygon
        
    Return:
        Nested list of polygons with coordinates
    """
    
    str_polygons = [poly.replace('(','') for poly in re.findall(r'\((.*?)\)', wkt_str)]

    polygons=[]
    for str_polygon in str_polygons:
        coordinates=[]
        for coordinate in str_polygon.split(','):
            coordinates.append(coordinate.strip())
        polygons.append(coordinates)
    
    return polygons

In [3]:
def close_line_string(polygons):
    
    """Close polygons if aren't closed
    
    Args:
        polygons (read_wkt): Nested list of polygons from read_wkt function
        
    Return:
    
        WKT polygon with closed linestring
    """
    # Check if the polygon is not closed
    
    res = []
    for polygon in polygons:
        if polygon[0]!=polygon[-1]:
            polygon += [polygon[0]]
            res.append(polygon)
        else:
            res.append(polygon)

    return f"POLYGON {res}".replace("'",'').replace('[','(').replace(']',')')

In [4]:
def filter_non_valid(df):
    """Filter non valid geometries due to lack of coordinates"""
    
    # Count number of pair-coordinates
    
    count_df = df.copy()
    
    count_df['GEOMETRIA'] = count_df['GEOMETRIA'].apply(read_wkt)
    count_df['COUNT'] = count_df['GEOMETRIA'].apply(lambda x: len(x[0]))
    count_df = count_df[count_df.COUNT>=4]
    
    return count_df

In [5]:
df = pd.read_csv('csv/Prueba_programacion.csv', delimiter=';')
df = filter_non_valid(df)
df;

In [6]:
# Fix unclosed polygons
df['GEOMETRIA'] = df['GEOMETRIA'].apply(close_line_string)

In [7]:
gdf = gpd.GeoDataFrame(df)
gdf['geometry'] = gdf['GEOMETRIA'].apply(loads)
gdf = gdf.set_crs('EPSG:4326')

In [8]:
# Create a convex hull to every polygon to fix its geometry
convex_gdf = gdf.copy()
convex_gdf.geometry = gdf.geometry.convex_hull

In [9]:
# Save raw shapefile
gdf.to_file(driver='ESRI Shapefile', filename='shp/polygons.shp')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [10]:
# Save convex hull shapefile
convex_gdf.to_file(driver='ESRI Shapefile', filename='shp/polygons_convexed.shp')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
